# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [2]:
# %load_ext sql
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

# pgconfig = {
#     'host': 'db',
#     'port': os.environ['PG_PORT'],
#     'database': os.environ['PG_DATABASE'],
#     'user': os.environ['PG_USER'],
#     'password': os.environ['PG_PASSWORD'],
# }
# dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# # pd.read_sql用のコネクタ
# conn = psycopg2.connect(**pgconfig)
# # pd.to_sql用のcreate engine
# engine = create_engine(dsl)
# # MagicコマンドでSQLを書くための設定
# %sql $dsl

# df_customer = pd.read_sql(sql='select * from customer', con=conn)
# df_category = pd.read_sql(sql='select * from category', con=conn)
# df_product = pd.read_sql(sql='select * from product', con=conn)
# df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
# df_store = pd.read_sql(sql='select * from store', con=conn)
# df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\iiats\Anaconda3\envs\Python36-gpu\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [4]:
df_category.head()

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,4,惣菜,401,御飯類,40101,弁当類
1,4,惣菜,401,御飯類,40102,寿司類
2,4,惣菜,402,佃煮類,40201,魚介佃煮類
3,4,惣菜,402,佃煮類,40202,海草佃煮類
4,4,惣菜,402,佃煮類,40203,野菜佃煮類


In [5]:
df_product.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0


In [6]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [7]:
df_store.head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


In [8]:
df_geocode.head()

,postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
0,060-0000,北海道,札幌市中央区,NaN,NaN,NaN,北海道札幌市中央区,141.34103,43.05513
1,064-0941,北海道,札幌市中央区,旭ケ丘,NaN,NaN,北海道札幌市中央区旭ケ丘,141.31972,43.04223
2,060-0042,北海道,札幌市中央区,大通西,NaN,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102
3,060-0042,北海道,札幌市中央区,大通西,NaN,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080
4,060-0042,北海道,札幌市中央区,大通西,NaN,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086


# 演習問題

---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [9]:
len(df_receipt)

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [41]:
len(df_receipt['customer_id'].unique())

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [42]:
df_receipt.groupby('store_cd').agg({'amount':'sum', 'quantity':'sum'}).reset_index()

,store_cd,amount,quantity
0,S12007,638761,2099
1,S12013,787513,2425
2,S12014,725167,2358
3,S12029,794741,2555
4,S12030,684402,2403
5,S13001,811936,2347
6,S13002,727821,2340
7,S13003,764294,2197
8,S13004,779373,2390
9,S13005,629876,2004


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [43]:
df_receipt.groupby('customer_id').sales_ymd.max().reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [44]:
df_receipt.groupby('customer_id').agg({'sales_ymd':'min'}).head(10)

,sales_ymd
customer_id,
CS001113000004,20190308
CS001114000005,20180503
CS001115000010,20171228
CS001205000004,20170914
CS001205000006,20180207
CS001211000025,20190322
CS001212000027,20170127
CS001212000031,20180906
CS001212000046,20170811
